Issues in line 0, 113281

In [1]:
from rdflib import Graph, Literal, RDF, URIRef,Namespace, BNode
from rdflib.namespace import FOAF , XSD,OWL,SKOS,RDFS

lmss = Namespace("http://lmss.sali.org/")
# Create a Graph
g = Graph()
# Parse in an RDF file hosted on the Internet
g.parse("SALI_new\SALI_new.owl")






ModuleNotFoundError: No module named 'rdflib'

In [163]:
#Python wraper for classes in ontology
class OWLClass:
    node_label = OWL.Class.n3(namespace_manager=g.namespace_manager).replace(":","_")
    
    def __init__(self, iri, label, defination):
        self.iri = iri
        self.label = label
        self.defination = defination
        self.parents = []
        self.relations = []

class Relation:
    def __init__(self, iri, label, to_iri):
        self.iri = iri
        self.label = label
        self.to_iri = to_iri

In [164]:
#FILL CLASS DETAILS 
onto_classes = g.subjects(RDF.type, OWL.Class)
classes = []
for c in onto_classes:
    
    label = g.value(c, RDFS.label)
    if label is None:
        label = c.split("#")[-1]
        
    
    classes.append(
        OWLClass( c,label, g.value(c, SKOS.definition))
        )
    


In [165]:
#FILL RELATIONSHIPS DETAILS AND PARENTS

for clas in classes:
    
    parents = g.objects(clas.iri, RDFS.subClassOf)
    
    for p in list(parents):
        
        if type(p) == BNode:
                
            what = g.value(p, OWL.onProperty)
            where = g.value(p, OWL.someValuesFrom)

            if what is None and where is None:
                continue

    
            if what.rsplit('/', 1)[0]+'/' == str(lmss):
                label=g.value(what, RDFS.label)
            else :   
                label=what.n3(namespace_manager=g.namespace_manager)
            
            clas.relations.append(
                Relation(what,label,where)
                )
            
        elif type(p) == URIRef:
            clas.parents.append(str(p))
            clas.relations.append(
                Relation(
                    RDFS.subClassOf,RDFS.subClassOf.n3(namespace_manager=g.namespace_manager),str(p)
                    )
                )
                

In [166]:
judge = classes[2]
for rel in judge.relations:
    print(rel.iri,rel.label,rel.to_iri)
    

http://www.w3.org/2000/01/rdf-schema#subClassOf rdfs:subClassOf http://lmss.sali.org/RBOjgvcq6Z33XxMhTxWiiDS


In [167]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "Dhananjay@10"

def create_nodes(tx, owl_classes):
    for owl_class in owl_classes:
        query = (
            "CREATE (:{} {{iri: $iri, label: $label, definition: $definition}})"
            .format(OWLClass.node_label)
        )
        
        
        #print the query which will be executed
        #print(query)
        tx.run(query, iri=owl_class.iri, label=owl_class.label, definition=owl_class.defination)


def create_relationships(tx, owl_classes):
    for owl_class in owl_classes:
        
        for relation in owl_class.relations:
            label = relation.label.replace(":", "_")

            
            query = (
                f"MATCH (a:{OWLClass.node_label} {{iri: '{owl_class.iri}'}}), "
                f"(b:{OWLClass.node_label} {{iri: '{relation.to_iri}'}}) "
                f"CREATE (a)-[:{label} {{ iri: '{relation.iri}'}} ]->(b)"
            )

            try:
                tx.run(query)
            except Exception as e:
                print(query)
                
        
def delete_all_nodes(tx):
    query = "MATCH (n) DETACH DELETE n"
    tx.run(query)
    



with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        #session.write_transaction(create_nodes, classes)
        session.write_transaction(create_relationships, classes)
        #session.write_transaction(delete_all_nodes)

C:\Users\dhana\AppData\Local\Temp\ipykernel_3844\40082491.py:44: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationships, classes)


In [168]:
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        #session.write_transaction(create_nodes, classes)
        #session.write_transaction(create_relationships, classes)
        #session.write_transaction(delete_all_nodes)

SyntaxError: incomplete input (2827926030.py, line 5)

['Mr. Milind Raj ', 'Mr. S.R.S. Gil', 'Mr. Lalit Belwal', 'Mr. Ashish Belwal']


Government Official


In [ ]:
from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import FOAF , XSD,OWL,SKOS,RDFS

# Create a Graph
g = Graph()

# Parse in an RDF file hosted on the Internet
g.parse("main.owl")

q = """
    PREFIX xml: <http://www.w3.org/XML/1998/namespace>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX v1: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX lmss: <http://lmss.sali.org/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX lmss1: <http://lmss.sali.org/2506>
    PREFIX schema: <http://schema.org/>
    PREFIX ontology: <http://www.geonames.org/ontology#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>


    SELECT ?r ?s ?t ?l
    WHERE {
        ?p rdf:type owl:Class .
        ?p rdfs:label "Judge" .
        ?p rdfs:subClassOf ?r .
        ?r rdf:type owl:Restriction .
        ?r owl:onProperty ?t.
        ?r owl:someValuesFrom ?s.
    }
"""
for r in g.query(q):
    print(r["r"])
    print(URIRef(r["s"]).n3(g.namespace_manager))
    print(URIRef(r["t"]).n3(g.namespace_manager))
    print(r["l"])

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")
    
    
    print(subj, pred, obj)

    

# Print the number of "triples" in the Graph
# print(f"Graph g has {len(g)} statements.")
# print(OWL.Class)



